In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf
from tensorflow.python.framework import ops
from time import time
from tensorflow import keras
from collections import deque
from tf2.Kfocusingtf2 import FocusedLayer1D
from scipy.integrate import trapz

# Plotting

In [2]:
def showTrend(scores, name=''):
    x = []
    y = []
    for i in range(0, len(scores)):
        x.append(int(i))
        y.append(int(scores[i]))

    plt.plot(x, y, label=name+" score per run")

    trend_x = x[1:]
    z = np.polyfit(np.array(trend_x), np.array(y[1:]), 1)
    p = np.poly1d(z)
    plt.plot(trend_x, p(trend_x), linestyle="-.",  label=name+" trend")


    plt.xlabel("episodes")
    plt.ylabel("scores")
    plt.legend(loc="upper left")
    plt.savefig('./graphs/'+nameFormat(name)+'_trend.png', bbox_inches="tight")

In [3]:
def statistic(scores, choices):
    meanByChoice = lambda lst, key: lst.count(key)/len(lst)

    print('Average Score: ',sum(scores)/len(scores))

    for i in set(choices):
        print(f"choice {i}:{meanByChoice(choices,i)}")

In [4]:
def lossComparison(name, results):
    plt.plot(results,  label=name+" loss")
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(loc="upper left")
    plt.savefig('./graphs/'+nameFormat(name)+'_loss_grahp.png', bbox_inches="tight")

In [5]:
nameFormat = lambda name:name+"-{}".format(int(time()))

# Neural Network Model

In [6]:
def build_model(N
                ,mode='dense'
                ,optimizer_s='adam'
                ,input_shape=None
                ,output_size=None
                ,savedModel=None):

    keras.backend.clear_session()

    if optimizer_s == 'SGDwithLR':
        pass #optimizer = SGDwithLR(lr_dict, mom_dict,decay_dict,clip_dict)
    elif optimizer_s == 'AdamwithCli':
        pass #optimizer = AdamwithClip()
    elif optimizer_s=='RMSpropwithClip':
        pass #optimizer = RMSpropwithClip(lr=0.001, rho=0.9, epsilon=None, decay=0.0,clips=clip_dict)
    elif optimizer_s=='adam':
        optimizer = keras.optimizers.Adam(lr=learning_rate)
    else:
        pass #optimizer = SGD(lr=0.01, momentum=0.9)
    
    if savedModel == None:
        if mode=='dense':
            layer = keras.layers.Dense(N[1], activation="elu")
        elif mode=='focused':
            layer = FocusedLayer1D(N[1]
                                   ,name='focus-1'
                                   ,activation='elu'
                                   ,init_sigma=0.25)
            # tried values:
            # .25, .5, .3, .2, .01, .2

        model = keras.models.Sequential([
            keras.layers.Dense(N[0], activation="elu", input_shape=input_shape),
            layer,
            keras.layers.Dense(output_size)
        ])
    else:
        try:
            restored_keras_model = keras.models.load_model(savedModel, custom_objects={'FocusedLayer1D': FocusedLayer1D})
        except:
            restored_keras_model = keras.models.load_model(savedModel)

    print(model.summary())

    return model, optimizer

# Creating Data and Test Run on Environment

In [7]:
class DQNAgent:
    def __init__(self
                 ,envName # the gym environment will run by agent
                 ,mode # the layer selection you may choice focused or dense
                 ,optimizer # optimizer selction will use on a model
                 ,seed=42 # random initializer
                 ,N=None # each layers neuron count
                 ,savedModel=None):

        # initialize N with 32 neuron each count by default
        if N == None:
            N = [32,32]
            
        if savedModel:
            savedModel = envName + '-' + mode + '.h5'

        self.env = gym.make(envName)
        self.env.seed(seed)
        self.input_shape = self.env.observation_space.shape

        # Discretized DQNAgent action space
        self.is_discrete = type(self.env.action_space) == gym.spaces.discrete.Discrete
        if not self.is_discrete:
            shape = self.env.action_space.shape
            low = self.env.action_space.low
            high = self.env.action_space.high
            self.env.action_space.n = 10
            self.actions = np.linspace(low, high, 10, dtype="float32")
            print('The environment is not discrete')
            print('The action space: ', self.actions.reshape(1,-1)[0])
        
        self.output_size = self.env.action_space.n

        self.envName = envName
        self.mode = mode
        
        self.replay_memory = deque(maxlen=2000)
        
        self.scores = []
        self.choices = []
        self.metrics = []
        self.best_score_index = 0
        self.best_score = 0
        self.score = 0
        
        self.epsilon = 0.95
        
        self.model, self.optimizer = build_model(N=N
                                                 ,mode=mode
                                                 ,optimizer_s=optimizer
                                                 ,input_shape=self.input_shape
                                                 ,output_size=self.output_size
                                                 ,savedModel=savedModel)
        
    def train(self):
        for episode in range(EPISODES):
            score = 0
            best_score_index = 0
            obs = self.env.reset()
            for step in range(ITERATIONS):
                self.epsilon = max(1 - episode / 500, 0.01)

                obs, reward, done, action = self.play_one_step(obs)

                score += reward

                if done:
                    break

            self.scores.append(score)

            if step > best_score_index:
                best_weights = self.model.get_weights()
                best_score_index = step

            print(f"\rEpisode: {episode+1}, Steps: {step+1}, eps: {self.epsilon}", end="")
            if episode > 50:
                lossValue= self.training_step()
                self.metrics.append(lossValue)

        self.model.set_weights(best_weights)
        
        self.saveModel()

    def play_one_step(self, state):
        state = self.setAxis(state)
        action = self.epsilon_greedy_policy(state)
        
        self.choices.append(action)
        
        next_state, reward, done, _ = self.env.step(self.get_action(action))

        self.replay_memory.append((state, self.isList(action), self.isList(reward), self.setAxis(next_state), self.isList(done)))
        return next_state, reward, done, action

    def get_action(self, action):
        if not self.is_discrete:
            action = [self.actions[action]]
        return action

    def setAxis(self, state):
        if not self.is_discrete:
            state = state.reshape(1,-1)[0]
        return state
    
    def isList(self, obj):
        try:
            obj = obj[0]
        finally:
            return obj

    def epsilon_greedy_policy(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.output_size)
        else:
            Q_values = self.model.predict(state[np.newaxis])

            return np.argmax(Q_values[0])
        
    def training_step(self):
        states, actions, rewards, next_states, dones = self.sample_experiences()

        next_Q_values = self.model.predict(next_states)
        max_next_Q_values = np.max(next_Q_values, axis=1)

        target_Q_values = (rewards +
                           (1 - dones) * discount_rate * max_next_Q_values)
        target_Q_values = target_Q_values.reshape(-1, 1)

        mask = tf.one_hot(actions, self.output_size)
        
        with tf.GradientTape() as tape:
            all_Q_values = self.model(states)
            Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
            loss = tf.reduce_mean(loss_fn(target_Q_values, Q_values))

        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

        if self.mode=='focused':
            self.clipcallBack('Sigma', (0.05,1.0))
            self.clipcallBack('Mu', (0.0,1.0))

        return float(loss)
    
    def clipcallBack(self, varname, clips):
        all_weights = self.model.trainable_weights

        for i,p in enumerate(all_weights):
            # print(p.name)
            if (p.name.find(varname)>=0):
                pval = p.numpy()
                clipped = np.clip(pval,clips[0],clips[1])
                p.assign(clipped)
                # print("Clipped", p.name)
                
    def sample_experiences(self):
        indices = np.random.randint(len(self.replay_memory), size=batch_size)
        batch = [self.replay_memory[index] for index in indices]
        states, actions, rewards, next_states, dones = [
            np.array([experience[field_index] for experience in batch])
            for field_index in range(5)]
        return states, actions, rewards, next_states, dones
    
    def saveModel(self):
        self.model.save('./models/{}-{}-{}.h5'.format(self.envName, self.mode, int(time())))
        path = './results/'+self.envName+'/'
        metrics = path+'{}-{}-metrics-{}.npy'.format(self.envName, self.mode, int(time()))
        np.save(metrics, np.array(self.metrics))
        scores = path+'{}-{}-scores-{}.npy'.format(self.envName, self.mode, int(time()))
        np.save(scores, np.array(self.scores))
        replay = path+'{}-{}-replay-{}.npy'.format(self.envName, self.mode, int(time()))
        np.save(replay, np.array(self.replay_memory))

    def testModel(self, render = True):
        for episode in range(EPISODES):
            currentState = self.setAxis(self.env.reset())

            print("============================================")

            rewardSum=0
            for iteration in range(200):
                if render:
                    self.env.render()

                action = np.argmax(self.model.predict(currentState)[0])

                new_state, reward, done, info = self.env.step(self.get_action(action))

                new_state = self.setAxis(new_state)

                currentState = new_state

                rewardSum += self.isList(reward)
                if done:
                    print("Episode finished after {} timesteps reward is {}".format(t+1,rewardSum))
                    break

# Credentials

In [8]:
# random initializers
mpl.style.use('default')
gym.logger.set_level(40)
np.random.seed(42)
ops.reset_default_graph()
tf.random.set_seed(42)

# loss function initializer
loss_fn = keras.losses.mean_squared_error

batch_size = 32 # sample size will driven to predict every episode
discount_rate = 0.95 # determines the importance of future rewards for q value
learning_rate = 1e-3 # determines to what extent newly acquired information overrides old information

# Test Model

In [ ]:
envName = 'CartPole-v1'  #@param ["CartPole-v1", "Acrobot-v1", "Pendulum-v0", "MountainCarContinuous-v0"] {type:"string"}
# how many episodes will run
EPISODES = 20 #@param {type: "number"}
# how many steps will be taken by a episode
ITERATIONS = 200 #@param {type: "number"}

mode = 'dense'#@param ["dense", "focused"] {type:"string"}

agent = DQNAgent(envName=envName
                 ,mode=mode
                 ,optimizer='adam'
                 ,savedModel=True)

agent.testModel()
print('Area Under Curve',trapz(agent.scores))

statistic(scores=agent.scores
          ,choices=agent.choices)

plt.figure(figsize=(20, 15))
# results
showTrend(scores=agent.scores, name=envName)